In [1]:
# Add the new path
import sys
new_path2 = "/home/michele/code/michele_mmdet3d/"
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)

# from mmdet3d.apis import LidarDet3DInferencer
# import os
# import torch
# import os
# import numpy as np
# import time
# from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
#                                                 LoadPointsFromFile)
# from mmdet3d.models.data_preprocessors.data_preprocessor import \
#     Det3DDataPreprocessor
# import onnxruntime as ort
# from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

In [2]:
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                 LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs
import os 

pcl = '/home/michele/code/michele_mmdet3d/data/minerva_polimove/training/velodyne'
inputs = dict(points=pcl)


def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = [inputs["points"] + "/" + s for s in os.listdir(inputs["points"])]
    
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input


In [ ]:
prep_input = prepare_input({"points": pcl}, batch_size=1)
print()
print(prep_input)

# Inferencer (half)

In [6]:
model = inferencer.model


In [ ]:
import time 
import torch

torch.manual_seed(42)
s = time.time()

prep_input = prepare_input({"points": pcl}, batch_size=1) # Preprocess
res = model(prep_input["inputs"],prep_input["data_samples"], mode="tensor") # Forward
preds  = model.bbox_head.predict_by_feat(*res) # Postprocess

print("Time: {} ms".format((time.time()-s)*1000))
print(preds)

In [ ]:
inf_res = inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)
inf_res

Convert Manually the tensor to predictions

In [ ]:
from mmdet3d.models.dense_heads.anchor3d_head import Anchor3DHead

# bbox_head=dict(
#     type='Anchor3DHead',
#     num_classes=3,
#     in_channels=384,
#     feat_channels=384,
#     use_direction_classifier=True,
#     assign_per_class=True,
#     anchor_generator=dict(
#         type='AlignedAnchor3DRangeGenerator',
#         ranges=[
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -1.78, 69.12, 39.68, -1.78],
#         ],
#         sizes=[[0.8, 0.6, 1.73], [1.76, 0.6, 1.73], [3.9, 1.6, 1.56]],
#         rotations=[0, 1.57],
#         reshape_out=False),
#     diff_rad_by_sin=True,
#     bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder')
# )
# model.bbox_head
head = Anchor3DHead(
    num_classes=1,
    in_channels=384,
    feat_channels=384,
    use_direction_classifier=True,
    assign_per_class=True,
    anchor_generator=dict(
                type='AlignedAnchor3DRangeGenerator',
                ranges=[[0, -39.68, -1.78, 69.12, 39.68, -1.78]],
                sizes=[[3.9, 1.6, 1.56]],
                rotations=[0, 1.57],
                reshape_out=True),
    diff_rad_by_sin=True,
    bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder'),
    test_cfg=model.test_cfg
    )
# preds  = head.predict_by_feat(*res)
preds  = model.bbox_head.predict_by_feat(*res)
preds

In [ ]:
preds[0]["bboxes_3d"],pred_res[0]["bboxes_3d"],inf_res["predictions"][0]["bboxes_3d"]

# Model Analysis

In [3]:
model = inferencer.model
pcl_path = "../data/DonaSet/training/velodyne/"
model_path = "end2end.onnx"

In [4]:
def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

### Random

In [ ]:
import torch
import os
import time
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

pcl_path = "../data/DonaSet/training/velodyne/"
s = time.time()

inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
# Evaluate on validation data
model.eval()
with torch.no_grad():
    val_output = model(inputs,mode="tensor")
    print("Time: {} ms".format((time.time()-s)*1000))


In [ ]:
import onnx

onnx_model_path = "end2end.onnx"
m = onnx.load(onnx_model_path)
opset_version = m.opset_import[0].version if m.opset_import else None

print("ONNX Model Operator Set Version:", opset_version)

In [ ]:
import onnxruntime

ort_version = onnxruntime.__version__
print("ONNX Runtime Version:", ort_version)

print( onnxruntime.get_device()  )

In [ ]:

import onnx
import onnxruntime as ort
import numpy as np

model_path = 'end2end.onnx'
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)

ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])


In [ ]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
s05 = time.time()
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
s3 = time.time()

print("Preprocessing took: {} ms".format((s05-s0)*1000))
print("Cpu conv took: {} ms".format((s1-s05)*1000))
print("Inference took: {} ms".format((s2-s1)*1000))
print("Postprocessing took: {} ms".format((s3-s2)*1000))
print("Total: {} ms".format((s3-s0)*1000))

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
s05 = time.time()
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
s1 = time.time()


## Inference time comparison

### Pytorch CPU

In [ ]:
model.cpu()
inputs = prepare_input({"points": pcl_path}, batch_size=1, device = "cpu")["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

### Pytorch GPU

In [ ]:
model.cuda()
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

### TensorRT Provider (ONNX Runtime)

In [ ]:
ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))


### CUDA Provider  (ONNX Runtime)

In [16]:
ort_sess = ort.InferenceSession(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

### CPU Provider  (ONNX Runtime)

In [21]:
ort_sess = ort.InferenceSession(model_path,providers=[ 'CPUExecutionProvider'])

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

#  TensorRT

In [ ]:
from mmdeploy.apis.inference import get_model

def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = [inputs["points"] + "/" + s for s in os.listdir(inputs["points"])]
    
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input



home_dir = '/home/michele/code/'
pcl_path = '/home/michele/code/michele_mmdet3d/data/minerva_polimove/training/velodyne'

model_cfg= home_dir + 'michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py'
deploy_cfg= home_dir + 'mmdeploy/configs/mmdet/detection/detection_tensorrt_dynamic-320x320-1344x1344.py'
backend_files= ['/home/michele/code/mmdeploy/mmdeploy_models/minerva_pointpillars/end2end.engine']
img= home_dir + 'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235584511788288.bin'
device= 'cuda'

tensorrt_model = get_model(
    model_cfg= home_dir + 'michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py',
    deploy_cfg= home_dir + 'mmdeploy/configs/mmdet/detection/detection_tensorrt_dynamic-320x320-1344x1344.py',
    backend_files= ['/home/michele/code/mmdeploy/mmdeploy_models/minerva_pointpillars/end2end.engine'],
    img= home_dir + 'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235584511788288.bin',
    device= 'cuda'
)

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]

res = tensorrt_model(inputs, None, mode="predict")
preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])